In [2]:
import tensorflow as tf

from srcnn.dataset import prepare_input_image, prepare_output_image
from srcnn.plot import plot_predictions, display_psnr

import os
import glob

In [8]:
def get_padding(model):
    sample_size = 100
    sample = tf.zeros([sample_size, sample_size, 3], dtype=tf.uint8)
    sample_out_size, _, _ = model.predict(tf.sta[sample])[0].get_shape()
    padding = (sample_size - sample_out_size) // 2

In [9]:
model_dir='models'
model_path = max(glob.iglob(os.path.join(model_dir, "*.h5")), key=os.path.getctime)
restored_model = tf.keras.models.load_model(model_path)
padding = get_padding(restored_model)
psnr = []

for filename in tf.io.matching_files("datasets/Test/Set5/*.bmp"):
    image_string = tf.io.read_file(filename)
    image = tf.image.decode_bmp(image_string, channels=3)
    h, w, _ = image.get_shape()
    
    input = prepare_input_image(image, h, w, scale=4)
    gt = prepare_output_image(image, h, w, h - 2*padding, w - 2*padding)
    
    short_name = os.path.split(f"{filename}")[1]
    last_index = short_name.find("_")
    short_name = short_name[:last_index]
    
    # can't use plot_predictions on all images since they have different sizes
    psnr = psnr + plot_predictions(restored_model, [short_name], [input], [gt])
    
    
display_psnr(psnr)

ValueError: Input 0 of layer sequential is incompatible with the layer: expected ndim=4, found ndim=3. Full shape received: [None, 100, 3]